<a href="https://colab.research.google.com/github/if001/redpajama_dolly_ja/blob/master/redpajama_dolly_ja.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## setup

In [1]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [1]:
!git clone https://github.com/if001/redpajama_dolly_ja.git
repo_dir='/content/redpajama_dolly_ja'
%cd $repo_dir

Cloning into 'redpajama_dolly_ja'...
remote: Enumerating objects: 156, done.
remote: Counting objects: 100% (156/156), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 156 (delta 62), reused 137 (delta 43), pack-reused 0
Receiving objects: 100% (156/156), 20.81 KiB | 4.16 MiB/s, done.
Resolving deltas: 100% (62/62), done.
/content/redpajama_dolly_ja


In [2]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-o9n5qzkm
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-o9n5qzkm
  Resolved https://github.com/huggingface/peft.git to commit b1059b73aab9043b118ff19b0cf96263ea86248a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 87.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 k

In [5]:
%cd $repo_dir
!git pull

/content/redpajama_dolly_ja
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 1), reused 3 (delta 1), pack-reused 0
Unpacking objects: 100% (3/3), 435 bytes | 435.00 KiB/s, done.
From https://github.com/if001/redpajama_dolly_ja
   0c1b59d..5317b3a  master     -> origin/master
Updating 0c1b59d..5317b3a
Fast-forward
 fine_tuning.py | 11 +++++++----
 1 file changed, 7 insertions(+), 4 deletions(-)


In [3]:
#@title Google Drive Options { display-mode: "form" }
save_models_to_drive = True #@param {type:"boolean"}
drive_mount = '/content/drive' #@param {type:"string"}
output_dir = 'redpajama_dolly_ja' #@param {type:"string"}
# tuned_model_name = 'ja' #@param {type:"string"}

import os
if save_models_to_drive:
    from google.colab import drive
    drive.mount(drive_mount)
    output_path = f"{drive_mount}/MyDrive/models/{output_dir}" if save_models_to_drive else f"/content/{output_dir}"
else:
    output_path = "/content"

#tuned_model_dir = f"{output_path}/{tuned_model_name}"
#os.makedirs(tuned_model_dir, exist_ok=True)
#os.makedirs(f"{output_path}/base_models/", exist_ok=True)

print(f"Saving models to {output_path}")

Mounted at /content/drive
Saving models to /content/drive/MyDrive/models/redpajama_dolly_ja


## check 


In [5]:
from transformers import AutoModelForCausalLM
import torch

model_name = "togethercomputer/RedPajama-INCITE-Base-3B-v1"

model = AutoModelForCausalLM.from_pretrained(model_name, 
                                            device_map='auto', 
                                            torch_dtype=torch.float16)
for v in model.state_dict().items():
  print(v[0])

gpt_neox.embed_in.weight
gpt_neox.layers.0.input_layernorm.weight
gpt_neox.layers.0.input_layernorm.bias
gpt_neox.layers.0.post_attention_layernorm.weight
gpt_neox.layers.0.post_attention_layernorm.bias
gpt_neox.layers.0.attention.bias
gpt_neox.layers.0.attention.masked_bias
gpt_neox.layers.0.attention.rotary_emb.inv_freq
gpt_neox.layers.0.attention.query_key_value.weight
gpt_neox.layers.0.attention.query_key_value.bias
gpt_neox.layers.0.attention.dense.weight
gpt_neox.layers.0.attention.dense.bias
gpt_neox.layers.0.mlp.dense_h_to_4h.weight
gpt_neox.layers.0.mlp.dense_h_to_4h.bias
gpt_neox.layers.0.mlp.dense_4h_to_h.weight
gpt_neox.layers.0.mlp.dense_4h_to_h.bias
gpt_neox.layers.1.input_layernorm.weight
gpt_neox.layers.1.input_layernorm.bias
gpt_neox.layers.1.post_attention_layernorm.weight
gpt_neox.layers.1.post_attention_layernorm.bias
gpt_neox.layers.1.attention.bias
gpt_neox.layers.1.attention.masked_bias
gpt_neox.layers.1.attention.rotary_emb.inv_freq
gpt_neox.layers.1.attention.q

## train

In [6]:
%cd $repo_dir
!python fine_tuning.py \
--out_dir $output_path \
--out_lora_dir $output_path \
--batch_size=1 \
--grad_ac=16 \
--epoch=1 \
--lora

/content/redpajama_dolly_ja
2023-05-09 01:28:31.515982: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/me

## infarence

In [4]:
import sys
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig

from peft import PeftModel


lora_weights = f"{output_path}"

base_model = "togethercomputer/RedPajama-INCITE-Base-3B-v1"
tokenizer = AutoTokenizer.from_pretrained(base_model)

device = None

if device == "cuda":
    model = AutoModelForCausalLM.from_pretrained(
                base_model,
                load_in_8bit=True,
                torch_dtype=torch.float16,
                device_map="auto",
            )
    model = PeftModel.from_pretrained(
            model,
            lora_weights,
            torch_dtype=torch.float16,
        )
else:
    model = AutoModelForCausalLM.from_pretrained(
            base_model, device_map={"": device}, low_cpu_mem_usage=True
    )
    model = PeftModel.from_pretrained(
            model,
            lora_weights,
            device_map={"": device},
        )
    
model.eval()

def gen_prompt(inp):
    PROMPT_FORMAT = '以下は、あるタスクを記述した指示です。質問に対する適切な回答を書きなさい。\n\n### 指示:\n{instruction}\n\n### 応答:\n'
    prompt = PROMPT_FORMAT.format(instruction=inp)
    return prompt


def evaluate(
        instruction,
        input=None,
        temperature=0.1,
        top_p=0.75,
        top_k=40,
        num_beams=4,
        max_new_tokens=128,
        stream_output=False,
        **kwargs,
    ):
    # prompt = prompter.generate_prompt(instruction, input)
    prompt = gen_prompt(instruction)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to(device)
    generation_config = GenerationConfig(
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            num_beams=num_beams,
            **kwargs,
        )
        
    with torch.no_grad():
        generation_output = model.generate(
                input_ids=input_ids,
                generation_config=generation_config,
                return_dict_in_generate=True,
                output_scores=True,
                max_new_tokens=max_new_tokens,
        )
        s = generation_output.sequences[0]
        output = tokenizer.decode(s)
        print(output)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:33: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/peft/utils/config.py:106 in from_pretrained              │
│                                                                                                  │
│   103 │   │   │   config_file = os.path.join(path, CONFIG_NAME)                                  │
│   104 │   │   else:                                                                              │
│   105 │   │   │   try:                                                                           │
│ ❱ 106 │   │   │   │   config_file = hf_hub_download(pretrained_model_name_or_path, CONFIG_NAME   │
│   107 │   │   │   except Exception:                                                              │
│   108 │   │   │   │   raise ValueError(f"Can't find '{CONFIG_NAME}' at '{pretrained_model_name   │
│   109                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_validators.py:112 in _inner_fn    │
│                                                                                                  │
│   109 │   │   │   kwargs.items(),  # Kwargs values                                               │
│   110 │   │   ):                                                                                 │
│   111 │   │   │   if arg_name in ["repo_id", "from_id", "to_id"]:                                │
│ ❱ 112 │   │   │   │   validate_repo_id(arg_value)                                                │
│   113 │   │   │                                                                                  │
│   114 │   │   │   elif arg_name == "token" and arg_value is not None:                            │
│   115 │   │   │   │   has_token = True                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_validators.py:160 in              │
│ validate_repo_id                                                                                 │
│                                                                                                  │
│   157 │   │   raise HFValidationError(f"Repo id must be a string, not {type(repo_id)}: '{repo_   │
│   158 │                                                                                          │
│   159 │   if repo_id.count("/") > 1:                                                             │
│ ❱ 160 │   │   raise HFValidationError(                                                           │
│   161 │   │   │   "Repo id must be in the form 'repo_name' or 'namespace/repo_name':"            │
│   162 │   │   │   f" '{repo_id}'. Use `repo_type` argument if needed."                           │
│   163 │   │   )                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': 
'/content/drive/MyDrive/models/redpajama_dolly_ja/checkpoint-500'. Use `repo_type` argument if needed.

During handling of the above exception, another exception occurred:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 15>:31                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/peft/peft_model.py:164 in from_pretrained                │
│                                                                                                  │
│   

In [ ]:
evaluate("色の三原色を教えてください")